In [1]:
import os
import torch

import numpy as np
import pandas as pd

from mlstm import mLSTM

os.environ["CUBLAS_WORKSPACE_CONFIG"] = ":4096:8"
torch.manual_seed(2022)
torch.use_deterministic_algorithms(True)

In [2]:
df = pd.read_csv("dataset/gbp-idr.csv")
prices = df.Close.values


def sliding_window(data, window_size=4, stride=1):
    return np.stack(
        [
            data[i : i + window_size]
            for i in range(0, data.shape[0] - window_size + 1, stride)
        ]
    )

In [3]:
window = sliding_window(prices, 257).astype(np.float32)
inputs, targets = window[:, :-1], window[:, -1]

size = inputs.shape[0]
while size % 128 > 0:
    size -= 1

train_size = size * 80 // 100

while train_size % 128 > 0:
    train_size -= 1

inputs_train, targets_train = inputs[:train_size], targets[:train_size]
inputs_eval, targets_eval = inputs[train_size:], targets[train_size:]
eval_size = inputs_eval.shape[0] // 2

inputs_valid, targets_valid = inputs_eval[eval_size:], targets_eval[eval_size:]
inputs_test, targets_test = inputs_eval[:eval_size], targets_eval[:eval_size]

valid_size = inputs_valid.shape[0]

while valid_size % 128 > 0:
    valid_size -= 1

inputs_valid, targets_valid = inputs_eval[:valid_size], targets_eval[:valid_size]

test_size = inputs_valid.shape[0]

while test_size % 128 > 0:
    test_size -= 1

inputs_valid, targets_valid = inputs_eval[:test_size], targets_eval[:test_size]

inputs_train, targets_train = torch.from_numpy(inputs_train), torch.from_numpy(
    targets_train
)
inputs_valid, targets_valid = torch.from_numpy(inputs_valid), torch.from_numpy(
    targets_valid
)
inputs_test, targets_test = torch.from_numpy(inputs_test), torch.from_numpy(
    targets_test
)

mean, std = inputs_train.mean(0), inputs_train.std(0)

dataset_train = torch.utils.data.TensorDataset(
    (inputs_train - mean) / std, targets_train
)
dataset_valid = torch.utils.data.TensorDataset(
    (inputs_valid - mean) / std, targets_valid
)
dataset_test = torch.utils.data.TensorDataset((inputs_test - mean) / std, targets_test)

dataloader_train = torch.utils.data.DataLoader(dataset_train, 128)
dataloader_valid = torch.utils.data.DataLoader(dataset_valid, 128)
dataloader_test = torch.utils.data.DataLoader(dataset_test, 128)

In [4]:
class TimeSeriesModel(torch.nn.Module):
    def __init__(self, input_size, hidden_size, n_mlstm=1):
        super().__init__()
        self.mlstm = mLSTM(input_size, hidden_size, n_mlstm)
        self.regressor = torch.nn.Sequential(
            torch.nn.Dropout(0.25), torch.nn.Linear(hidden_size, 1)
        )

    def forward(self, inputs, hidden_states=None):
        _, (h_n, c_n) = self.mlstm(inputs, hidden_states)

        return (h_n, c_n), self.regressor(h_n[-1])

In [5]:
model = TimeSeriesModel(256, 512).cuda()
optimizer = torch.optim.Adam(model.parameters(), 0.5)
scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(
    optimizer, len(dataloader_train) * 25
)

criterion = torch.nn.HuberLoss()

for epoch in range(100):
    losses_train, losses_valid = [], []
    hidden_state = None
    model.train()
    for inputs, targets in dataloader_train:
        inputs = inputs.cuda()
        targets = targets.cuda()

        (h_n, c_n), predictions = model(inputs, hidden_state)
        hidden_state = h_n[-1], c_n[-1]
        loss = criterion(predictions.squeeze(1), targets)
        loss.backward()

        optimizer.step()
        scheduler.step()
        optimizer.zero_grad()

        losses_train.append(loss.detach().item())

    hidden_state = None
    model.eval()
    for inputs, targets in dataloader_valid:
        inputs = inputs.cuda()
        targets = targets.cuda()

        with torch.no_grad():
            (h_n, c_n), predictions = model(inputs, hidden_state)
            hidden_state = h_n[-1], c_n[-1]
            loss = criterion(predictions.squeeze(1), targets)

        losses_valid.append(loss.item())

    print(f"Epoch {epoch+1}/100")
    print(f"Training Loss: {np.mean(losses_train)}")
    print(f"Validation Loss: {np.mean(losses_valid)}\n")

Epoch 1/100
Training Loss: 14522.623453776041
Validation Loss: 14784.06103515625

Epoch 2/100
Training Loss: 11353.149993896484
Validation Loss: 11634.990397135416

Epoch 3/100
Training Loss: 8453.906885782877
Validation Loss: 8394.091634114584

Epoch 4/100
Training Loss: 9146.588321685791
Validation Loss: 10695.666341145834

Epoch 5/100
Training Loss: 14041.235088348389
Validation Loss: 6998.41015625

Epoch 6/100
Training Loss: 10853.359436035156
Validation Loss: 3323.6253662109375

Epoch 7/100
Training Loss: 8199.039396921793
Validation Loss: 830.0427398681641

Epoch 8/100
Training Loss: 6815.795986175537
Validation Loss: 901.8446807861328

Epoch 9/100
Training Loss: 5592.1659418741865
Validation Loss: 907.3913523356119

Epoch 10/100
Training Loss: 4446.1663004557295
Validation Loss: 923.8132069905599

Epoch 11/100
Training Loss: 3437.0196380615234
Validation Loss: 915.4478098551432

Epoch 12/100
Training Loss: 2907.88316599528
Validation Loss: 945.8363444010416

Epoch 13/100
Trainin